# Finding Similar issues by semantics

In [1]:
!pip install sentence-transformers scikit-learn pandas

In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

## Loading github issues for transfomer library

In [3]:
pd.set_option('display.max_colwidth', None)

# Load dataset
df = pd.read_csv("gitissues.csv")  # update this path

#df[['title', 'comments']].sample(10)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2175 entries, 0 to 2174
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      2175 non-null   int64 
 1   html_url        2175 non-null   object
 2   title           2175 non-null   object
 3   comments        2175 non-null   object
 4   body            2174 non-null   object
 5   comment_length  2175 non-null   int64 
 6   text            2175 non-null   object
dtypes: int64(2), object(5)
memory usage: 119.1+ KB


In [7]:
df[['title', 'comments']].sample(10)

,title,comments
871,benchmarking against MMapIndexedDataset,Also I would be interested to know what data types `MMapIndexedDataset` supports. Is there some documentation somewhere ?
340,Saving Graph/Structured Data in Datasets,"Hi,\r\n\r\nyou can do the following to load this data into a `Dataset`:\r\n```python\r\nfrom datasets import Dataset\r\nexamples = [\r\n {\r\n ""name"": ""mike"",\r\n ""friends"": [\r\n ""tom"",\r\n ""lily""\r\n ],\r\n ""articles"": [\r\n {\r\n ""title"": ""aaaaa"",\r\n ""reader"": [\r\n ""tom"",\r\n ""lucy""\r\n ]\r\n }\r\n ]\r\n },\r\n {\r\n ""name"": ""tom"",\r\n ""friends"": [\r\n ""mike"",\r\n ""bbb""\r\n ],\r\n ""articles"": [\r\n {\r\n ""title"": ""xxxxx"",\r\n ""reader"": [\r\n ""tom"",\r\n ""qqqq""\r\n ]\r\n }\r\n ]\r\n }\r\n]\r\n\r\nkeys = examples[0].keys()\r\nvalues = [ex.values() for ex in examples]\r\ndataset = Dataset.from_dict({k: list(v) for k, v in zip(keys, zip(*values))})\r\n```\r\n\r\nLet us know if this works for you."
1377,Error loading ms_marco v2.1 using load_dataset(),There was a similar issue in #294 \r\nClearing the cache and download again the dataset did the job. Could you try to clear your cache and download the dataset again ?
349,Cached dataset not loaded,"> does it consider just the name or the actual code of the function\r\n\r\nIt looks at the name and the actual code and all variables such as recursively. It uses `dill` to do so, which is based on `pickle`.\r\nBasically the hash is computed using the pickle bytes of your function (computed using `dill` to support most python objects).\r\n\r\n> does it consider variables that are not passed explicitly as parameters to the functions (such as the processor here)\r\n\r\nYes it does thanks to recursive pickling."
834,[experiment] missing default_experiment-1-0.arrow,"Thank you for explaining that in a great way, @lhoestq \r\n\r\nSo the bottom line is that the `transformers` examples are broken since they don't do any of that. At least `run_seq2seq.py` just does `metric = load_metric(metric_name)`\r\n\r\nWhat test would you recommend to reliably reproduce this bug in `examples/seq2seq/run_seq2seq.py`?"
23,"In v1.4.1, all TIMIT train transcripts are ""Would such an act of refusal be useful?""","Hi @rcgale, thanks for reporting.\r\n\r\nPlease note that this bug was fixed on `datasets` version 1.5.0: https://github.com/huggingface/datasets/commit/a23c73e526e1c30263834164f16f1fdf76722c8c#diff-f12a7a42d4673bb6c2ca5a40c92c29eb4fe3475908c84fd4ce4fad5dc2514878\r\n\r\nIf you update `datasets` version, that should work.\r\n\r\nOn the other hand, would it be possible for @patrickvonplaten to update the [blog post](https://huggingface.co/blog/fine-tune-wav2vec2-english) with the correct version of `datasets`?"
1706,[Dataset] RACE dataset Checksums error,"I just cleared the cache an run it again. The error persists ):\r\n\r\n```\r\n nlp (master) $ rm -rf /Users/abarbosa/.cache/huggingface/\r\n nlp (master) $ python\r\nPython 3.8.5 (default, Aug 5 2020, 03:39:04)\r\n[Clang 10.0.0 ] :: Anaconda, Inc. on darwin\r\nType ""help"", ""copyright"", ""credits"" or ""license"" for more information.\r\n>>> import nlp\r\n>>> dataset = nlp.load_dataset(""race"")\r\nDownloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4.39k/4.39k [00:00<00:00, 661kB/s]\r\nDownloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.81k/1.81k [00:00<00:00, 644kB/s]\r\nUsing custom data configuration default\r\nDownloading and preparing dataset race/default (download: 84.52 MiB, generated: 132.61 MiB, post-processed: Unknown size, total: 217.13 MiB) to /Users/abarbosa/.cache/huggingface/datasets/race/default/0.1.0/5461327f1a83549ca0d845a3159c806d2baf4f8d0d8f7d657157ce7cdf3899c2...\r\nDownloading: 100%|████████████████████████████████████████████

In [10]:
df[0:1]

,Unnamed: 0,html_url,title,comments,body,comment_length,text
0,0,https://github.com/huggingface/datasets/issues/2945,Protect master branch,"@lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history).","After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n - Currently, simple merge commits are already disabled\r\n - I propose to disable rebase merging as well\r\n- ~~Protect the master branch from direct pushes (to avoid accidentally pushing of merge commits)~~\r\n - ~~This protection would reject direct pushes to master branch~~\r\n - ~~If so, for each release (when we need to commit directly to the master branch), we should previously disable the protection and re-enable it again after the release~~\r\n- [x] Protect the master branch only from direct pushing of **merge commits**\r\n - GitHub offers the possibility to protect the master branch only from merge commits (which are the ones that introduce all the commits from the feature branch into the master branch).\r\n - No need to disable/re-enable this protection on each release \r\n\r\nThis purpose of this Issue is to open a discussion about this problem and to agree in a solution.",64,"Protect master branch \n After accidental merge commit (91c55355b634d0dc73350a7ddee1a6776dbbdd69) into `datasets` master branch, all commits present in the feature branch were permanently added to `datasets` master branch history, as e.g.:\r\n- 00cc036fea7c7745cfe722360036ed306796a3f2\r\n- 13ae8c98602bbad8197de3b9b425f4c78f582af1\r\n- ...\r\n\r\nI propose to protect our master branch, so that we avoid we can accidentally make this kind of mistakes in the future:\r\n- [x] For Pull Requests using GitHub, allow only squash merging, so that only a single commit per Pull Request is merged into the master branch\r\n - Currently, simple merge commits are already disabled\r\n - I propose to disable rebase merging as well\r\n- ~~Protect the master branch from direct pushes (to avoid accidentally pushing of merge commits)~~\r\n - ~~This protection would reject direct pushes to master branch~~\r\n - ~~If so, for each release (when we need to commit directly to the master branch), we should previously disable the protection and re-enable it again after the release~~\r\n- [x] Protect the master branch only from direct pushing of **merge commits**\r\n - GitHub offers the possibility to protect the master branch only from merge commits (which are the ones that introduce all the commits from the feature branch into the master branch).\r\n - No need to disable/re-enable this protection on each release \r\n\r\nThis purpose of this Issue is to open a discussion about this problem and to agree in a solution. \n @lhoestq now the 2 are implemented.\r\n\r\nPlease note that for the the second protection, finally I have chosen to protect the master branch only from **merge commits** (see update comment above), so no need to disable/re-enable the protection on each release (direct commits, different from merge commits, can be pushed to the remote master branch; and eventually reverted without messing up the repo history)."


## Embedding the issues

In [8]:
def embed_issues(df, column_to_embed='title', model_name='all-MiniLM-L6-v2'):
    """
    Embeds the selected text column using SentenceTransformer.

    Parameters:
    - df: DataFrame containing the GitHub issues.
    - column_to_embed: Column name to embed ('title', 'body', etc.).
    - model_name: Pre-trained model name from sentence-transformers.

    Returns:
    - embeddings: numpy array of sentence embeddings.
    - model: the loaded SentenceTransformer model.
    """
    model = SentenceTransformer(model_name)
    texts = df[column_to_embed].fillna('').tolist()
    embeddings = model.encode(texts, convert_to_numpy=True)
    return embeddings, model

In [9]:
# Configuration
column = 'title'

# Step 1: Embed dataset
issue_embeddings, embed_model = embed_issues(df, column_to_embed=column)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
issue_embeddings[0:1]

array([[ 1.25918994e-02,  2.79929675e-02,  2.98907310e-02,
        -2.82609407e-02,  8.30339789e-02,  2.21100934e-02,
         5.24233794e-03,  1.80427507e-02,  2.48939302e-02,
         4.96568009e-02, -5.40389679e-02,  3.42968442e-02,
         7.10182860e-02, -4.21273634e-02, -2.66767144e-02,
         5.21195568e-02, -5.02375513e-02,  4.12489586e-02,
         3.15377973e-02, -4.57764678e-02, -2.44398788e-02,
        -5.04018320e-03, -7.71648139e-02,  3.12126447e-02,
        -6.78871349e-02, -7.86000937e-02, -7.00689200e-03,
        -2.02241645e-04, -2.91259517e-03, -9.64545161e-02,
         2.02910956e-02, -1.72545128e-02, -2.35775337e-02,
         7.72400796e-02, -3.75692435e-02, -2.09935172e-03,
         1.67593360e-02, -2.95889974e-02, -5.15630189e-03,
        -1.38420537e-02, -1.93502270e-02,  8.89721289e-02,
         2.37519536e-02, -6.59949780e-02,  3.86768044e-03,
         4.09940034e-02, -7.31461041e-04, -1.45085836e-02,
        -7.21288025e-02, -7.32265553e-03,  9.17877853e-0

## Finding similar issues

In [11]:
def search_similar_issues(df, embeddings, model, problem_description, k=5, column_to_embed='title'):
    """
    Finds top-k similar issues from the dataset.

    Parameters:
    - df: DataFrame with GitHub issues.
    - embeddings: Precomputed issue embeddings.
    - model: SentenceTransformer model used for embedding.
    - problem_description: Query string to compare against dataset.
    - k: Number of top similar issues to return.
    - column_to_embed: For display context, can be title or body.

    Returns:
    - DataFrame with top-k similar issues and similarity scores.
    """
    query_embedding = model.encode([problem_description], convert_to_numpy=True)
    sim_scores = cosine_similarity(query_embedding, embeddings)[0]
    top_k_idx = np.argsort(sim_scores)[-k:][::-1]

    result_df = df.iloc[top_k_idx].copy()
    result_df['similarity'] = sim_scores[top_k_idx]
    return result_df[['title', 'similarity']]

## Seach similar problems

In [12]:
k = 10

query = "caching configuration not working"

# Step 2: Search similar issues
similar = search_similar_issues(df,
                                issue_embeddings,
                                embed_model,
                                query,
                                k=k,
                                column_to_embed=column)

similar

,title,similarity
1424,Possible caching bug,0.672567
1757,Caching doesn't work for map (non-deterministic),0.620773
150,Missing cache file,0.613573
151,Missing cache file,0.613573
349,Cached dataset not loaded,0.578330
346,Cached dataset not loaded,0.578330
348,Cached dataset not loaded,0.578330
347,Cached dataset not loaded,0.578330
1507,Caching processed dataset at wrong folder,0.567681
1509,Caching processed dataset at wrong folder,0.567681
